# Music Predictor
Using this file to predict the genre of a given music file

In [6]:
from music_dealer import MusicDealer
from model import M_model
import os

WEIGHT_PATH = "../model/best_model_1.pt"
dealer = MusicDealer(WEIGHT_PATH, M_model)

# Sample Music Results

In [ ]:
data_path = '../sample_music'
for (dirpath, dirnames, filenames) in os.walk(data_path):
    for file in filenames:
        file_path = data_path + "/" + file
        if 'mp3' not in file_path:
            continue
        dealer.get_genre(file_path)
        print("*"*99)

Test on Kendrick Lamar - HUMBLE..mp3
Genre hiphop: 58.09%
Genre pop: 31.78%
Genre disco: 4.6%
Genre reggae: 2.94%
Genre blues: 2.6%
***************************************************************************************************
Test on Jackson Wang- Papillon.mp3
Genre pop: 60.93%
Genre reggae: 19.63%
Genre hiphop: 16.87%
Genre blues: 1.29%
Genre disco: 1.28%
***************************************************************************************************
Test on Adele - Hello.mp3
Genre pop: 61.9%
Genre blues: 19.15%
Genre country: 11.72%
Genre reggae: 3.87%
Genre hiphop: 2.67%
Genre metal: 0.7%
***************************************************************************************************
Test on Aimer - Brave Shine.mp3
Genre pop: 59.37%
Genre hiphop: 21.87%
Genre blues: 9.25%
Genre reggae: 3.51%
Genre rock: 2.78%
Genre country: 1.79%
Genre disco: 1.43%
***************************************************************************************************
Test on Adam Lambert - W